# .NET 6 Microsoft Orleans Hello World Demo

You need to install [.NET 6 SDK](https://dotnet.microsoft.com/download/dotnet/6.0) and [PowerShell 7+](https://learn.microsoft.com/powershell/scripting/install/installing-powershell), and also install [Ployglot Notebooks VSCode extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) for running this notebook.

First, we need to use PowerShell to build Orleans' Grain class library project:

In [2]:
#!pwsh
dotnet build ../src/Grains/HelloWorld.Grains.Greeting/HelloWorld.Grains.Greeting.csproj --nologo --no-self-contained --verbosity minimal

  Determining projects to restore...
  All projects are up-to-date for restore.
  HelloWorld.Interfaces.Hello -> G:\OrleansNet6HelloWorld\src\Shared\HelloWorld.Interfaces.Hello\bin\Debug\netstandard2.0\HelloWorld.Interfaces.Hello.dll
  HelloWorld.Grains.Greeting -> G:\OrleansNet6HelloWorld\src\Grains\HelloWorld.Grains.Greeting\bin\Debug\net6.0\HelloWorld.Grains.Greeting.dll

Build succeeded.
    0 Warning(s)
    0 Error(s)

Time Elapsed 00:00:01.82

Workload updates are available. Run `dotnet workload list` for more information.


Then load just built Assembly file:

In [4]:
#r "../src/Grains/HelloWorld.Grains.Greeting/bin/Debug/net6.0/HelloWorld.Grains.Greeting.dll"

Install following Nueget packages:

// base libraries for hosting an Orleans Host instance
- [Microsoft.Extensions.Hosting](https://www.nuget.org/packages/Microsoft.Extensions.Hosting/6.0.1)
- [Microsoft.Orleans.Server](https://www.nuget.org/packages/Microsoft.Orleans.Server/3.7.1)
- [Microsoft.Orleans.OrleansCodeGenerator](https://www.nuget.org/packages/Microsoft.Orleans.OrleansCodeGenerator/3.7.1)

// logging libraries to show messages on Jupyter Notebook
- [Microsoft.Extensions.Logging.Abstractions](https://www.nuget.org/packages/Microsoft.Extensions.Logging.Abstractions/6.0.4)
- [Microsoft.Extensions.Logging](https://www.nuget.org/packages/Microsoft.Extensions.Logging/6.0.0)
- [Serilog.Extensions.Logging](https://www.nuget.org/packages/Serilog.Extensions.Logging/3.1.0)
- [Serilog.Sinks.Console](https://www.nuget.org/packages/Serilog.Sinks.Console/4.1.0)

(Note: since .NET Interactive Kernel is not a pure C# project, [we **cannot** use the Build time code generation packages](https://docs.microsoft.com/en-us/dotnet/orleans/grains/code-generation#what-happens-during-build) but instead install `Microsoft.Orleans.OrleansCodeGenerator` nuget that do code generation when both client & silo starting up)

In [1]:
// base libraries for hosting an Orleans Host instance
#r "nuget: Microsoft.Extensions.Hosting, 6.0.1"
#r "nuget: Microsoft.Orleans.Server, 3.7.1"
#r "nuget: Microsoft.Orleans.OrleansCodeGenerator, 3.7.1"

// logging libraries to show messages on Jupyter Notebook
#r "nuget: Microsoft.Extensions.Logging.Abstractions, 6.0.4"
#r "nuget: Microsoft.Extensions.Logging, 6.0.0"
#r "nuget: Serilog.Extensions.Logging, 3.1.0"
#r "nuget: Serilog.Sinks.Console, 4.1.0"


Installed Packages Microsoft.Extensions.Hosting, 6.0.1 Microsoft.Extensions.Logging, 6.0.0 Microsoft.Extensions.Logging.Abstractions, 6.0.4 Microsoft.Orleans.OrleansCodeGenerator, 3.7.1 Microsoft.Orleans.Server, 3.7.1 Serilog.Extensions.Logging, 3.1.0 Serilog.Sinks.Console, 4.1.0

Using some namespace to reduce clutter code and enable hosting extension methods:

In [6]:
// Orleans Silo
using Microsoft.Extensions.Hosting;
using Orleans;
using Orleans.Hosting;
using Orleans.Runtime;
using Orleans.Configuration;

using Microsoft.Extensions.Logging;
using Serilog;

//Our Orleans Grain & Interface
using HelloWorld.Grains.Greeting;
using HelloWorld.Interfaces.Hello;

## Create Orleans Server (Silo Host)

Create [HostBuilder](https://learn.microsoft.com/dotnet/api/microsoft.extensions.hosting.hostbuilder?view=net-6.0) and properly configure it using [`UseOrleans()` extension method](https://learn.microsoft.com/dotnet/api/microsoft.extensions.hosting.generichostextensions.useorleans?view=orleans-3.0):


In [11]:
Log.Logger = new LoggerConfiguration().WriteTo.Console().CreateLogger();
var codeGenLoggerFactory = LoggerFactory.Create(logBuilder => logBuilder.AddSerilog());

const string DemoClusterId = @"demo-cluster";
const string DemoServiceId = @"JuypterNotebook HelloWorld";

var hostBuilder = new HostBuilder()
    .UseOrleans(siloBuilder =>
    {
        siloBuilder
            .UseLocalhostClustering()
            .Configure<ClusterOptions>(options =>
            {
                options.ClusterId = DemoClusterId;
                options.ServiceId = DemoServiceId;
            })
            .ConfigureApplicationParts(parts => { 
                parts.AddApplicationPart(typeof(HelloGrain).Assembly).WithCodeGeneration(codeGenLoggerFactory);
                parts.AddApplicationPart(typeof(IHelloGrain).Assembly).WithCodeGeneration(codeGenLoggerFactory);
            });
    });

Create Silo Host instance via `hostBuilder`, and start it up:

In [12]:
var host = hostBuilder.Build();
host.Start();

[20:50:42 INF] Generating code for assemblies: 
[20:50:42 INF] Runtime code generation for assemblies  HelloWorld.Grains.Greeting, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 344 milliseconds
[20:50:42 INF] Generating code for assemblies: 
[20:50:42 INF] Runtime code generation for assemblies  HelloWorld.Interfaces.Hello, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 321 milliseconds


## Create Orleans Client and call Hello World RPC method

Create [ClientBuilder](https://learn.microsoft.com/dotnet/api/orleans.clientbuilder?view=orleans-3.0) and configure it:

In [14]:
var clientBuilder = new ClientBuilder().UseLocalhostClustering()
    .Configure<ClusterOptions>(options =>
    {
        options.ClusterId = DemoClusterId;
        options.ServiceId = DemoServiceId;
    })
    .ConfigureApplicationParts(parts =>
    {
        // we only needs grain interface code gen on client side
        parts.AddApplicationPart(typeof(IHelloGrain).Assembly).WithCodeGeneration(codeGenLoggerFactory);
    });

[20:54:23 INF] Generating code for assemblies: 
[20:54:23 INF] Runtime code generation for assemblies  HelloWorld.Interfaces.Hello, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null took 454 milliseconds


Create Orleans client via `clientBuilder`, and call [`Connect()` async method](https://learn.microsoft.com/dotnet/api/orleans.iclusterclient.connect?view=orleans-3.0) to connect to Silo Host, then get an Orleans grain client side RPC instance via a defined identifier(we use *0* here):

In [15]:
var client = clientBuilder.Build();
await client.Connect();
var demoGrain = client.GetGrain<IHelloGrain>(0);

The `demoGrain` is the client grain RPC instance, call its `Task<string> SayHello(string greeting)` RPC method:

In [17]:
var greeting = await demoGrain.SayHello("Hello Orleans");
display("RPC return is: \"" + greeting + "\"");

RPC return is: "You said: 'Hello Orleans', I say: Hello!"

## Clean up

First, close connection between Orleans Client and Silo Host by calling [`Close()` async method](https://learn.microsoft.com/dotnet/api/orleans.iclusterclient.close?view=orleans-3.0) on Client side:

In [18]:
await client.Close();

Shutdown Silo Host:

In [19]:
await host.StopAsync();